# Create Word-Context Co-Occurrence Matrix

In [40]:
# import modules
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
import re
from nltk.stem import WordNetLemmatizer
import pickle
from scipy.io import savemat

In [2]:
# to convert contractions picked up by word_tokenize() into full words
contractions = {
    "n't": 'not',
    "'ve": 'have',
    "'s": 'is', # note that this will include possessive nouns
    'gonna': 'going to',
    'gotta': 'got to',
    "'d": 'would',
    "'ll": 'will',
    "'re": 'are',
    "'m": 'am',
    'wanna': 'want to'
}

# to convert nltk_pos tags to wordnet-compatible PoS tags
def convert_pos_wordnet(tag):
    tag_abbr = tag[0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
                
    if tag_abbr in tag_dict:
        return tag_dict[tag_abbr]

In [3]:
class ContextMatrix(TransformerMixin):
    
    # initialize class & private variables
    def __init__(self,
                 window_size = 4,
                 remove_stopwords = True,
                 add_start_end_tokens = True,
                 lowercase = False,
                 lemmatize = False,
                 pmi = False,
                 spmi_k = 1,
                 laplace_smoothing = 0,
                 pmi_positive = False,
                 sppmi_k = 1):
        
        """ Params:
                window_size: size of +/- context window (default = 4)
                remove_stopwords: boolean, whether or not to remove NLTK English stopwords
                add_start_end_tokens: boolean, whether or not to append <START> and <END> to the
                beginning/end of each document in the corpus (default = True)
                lowercase: boolean, whether or not to convert words to all lowercase
                lemmatize: boolean, whether or not to lemmatize input text
                pmi: boolean, whether or not to compute pointwise mutual information
                pmi_positive: boolean, whether or not to compute positive PMI
        """
        self.window_size = window_size
        self.remove_stopwords = remove_stopwords
        self.add_start_end_tokens = add_start_end_tokens
        self.lowercase = lowercase
        self.lemmatize = lemmatize
        self.pmi = pmi
        self.spmi_k = spmi_k
        self.laplace_smoothing = laplace_smoothing
        self.pmi_positive = pmi_positive
        self.sppmi_k = sppmi_k
        self.corpus = None
        self.clean_corpus = None
        self.vocabulary = None
        self.X = None
        self.doc_terms_lists = None
    
    def fit(self, corpus, y = None):
        
        """ Learn the dictionary of all unique tokens for given corpus.
        
            Params:
                corpus: list of strings
            
            Returns: self
        """
        self.corpus = corpus
        
        term_dict = dict()
        k = 0
        corpus_words = []
        clean_corpus = []
        doc_terms_lists = []
        detokenizer = TreebankWordDetokenizer()
        lemmatizer = WordNetLemmatizer()
        
        for text in corpus:
            text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
            
            words = word_tokenize(text)
            
            if self.remove_stopwords:
                clean_words = []
                for word in words:
                    if word.lower() not in set(stopwords.words('english')):
                        clean_words.append(word)
                words = clean_words
                
            if self.lowercase:
                clean_words = []
                for word in words:
                    clean_words.append(word.lower())
                
                words = clean_words
                
            if self.lemmatize:
                clean_words = []
                for word in words:
                    PoS_tag = pos_tag([word])[0][1]
                    
                    # to change contractions to full word form
                    if word in contractions:
                        word = contractions[word]

                    if PoS_tag[0].upper() in 'JNVR':
                        word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                    else:
                        word = lemmatizer.lemmatize(word)

                    clean_words.append(word)
                    
                words = clean_words
            
            # detokenize trick taken from this StackOverflow post:
            # https://stackoverflow.com/questions/21948019/python-untokenize-a-sentence
            # and NLTK treebank documentation:
            # https://www.nltk.org/_modules/nltk/tokenize/treebank.html
            text = detokenizer.detokenize(words)
            clean_corpus.append(text)
            
            [corpus_words.append(word) for word in words]
            
            if self.add_start_end_tokens:
                words = ['<START>'] + words + ['<END>']
            
            doc_terms_lists.append(words)
            
        self.clean_corpus = clean_corpus
        
        self.doc_terms_lists = doc_terms_lists
        
        corpus_words = list(set(corpus_words))
        
        if self.add_start_end_tokens:
            corpus_words = ['<START>'] + corpus_words + ['<END>']
        
        corpus_words = sorted(corpus_words)
        
        for el in corpus_words:
            term_dict[el] = k
            k += 1
            
        self.vocabulary = term_dict
        
        return self
        
    def transform(self, new_corpus = None, y = None):
        
        """ Compute the co-occurrence matrix for given corpus and window_size, using term dictionary
            obtained with fit method.
        
            Returns: term-context co-occurrence matrix (shape: target terms by context terms) with
            raw counts
        """
        num_terms = len(self.vocabulary)
        window = self.window_size
        X = np.full((num_terms, num_terms), self.laplace_smoothing)
        
        lemmatizer = WordNetLemmatizer()
        if type(new_corpus) != list:
            new_corpus = self.corpus
        
        for text in new_corpus:
            text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
            
            words = word_tokenize(text)
            
            if self.remove_stopwords:
                clean_words = []
                for word in words:
                    if word.lower() not in set(stopwords.words('english')):
                        clean_words.append(word)
                words = clean_words
                
            if self.lowercase:
                clean_words = []
                for word in words:
                    clean_words.append(word.lower())
                
                words = clean_words
                
            if self.lemmatize:
                clean_words = []
                for word in words:
                    PoS_tag = pos_tag([word])[0][1]
                    
                    # to change contractions to full word form
                    if word in contractions:
                        word = contractions[word]

                    if PoS_tag[0].upper() in 'JNVR':
                        word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                    else:
                        word = lemmatizer.lemmatize(word)

                    clean_words.append(word)
                    
                words = clean_words
                
            if self.add_start_end_tokens:
                words = ['<START>'] + words + ['<END>']
            
            for i in range(len(words)):
                target = words[i]
                
                # check to see if target word is in the dictionary; if not, skip
                if target in self.vocabulary:
                    
                    # grab index from dictionary
                    target_dict_index = self.vocabulary[target]
                    
                    # find left-most and right-most window indices for each target word
                    left_end_index = max(i - window, 0)
                    right_end_index = min(i + window, len(words) - 1)
                    
                    # loop over all words within window
                    # NOTE: this will include the target word; make sure to skip over it
                    for j in range(left_end_index, right_end_index + 1):
                        
                        # skip "context word" where the "context word" index is equal to the
                        # target word index
                        if j != i:
                            context_word = words[j]
                            
                            # check to see if context word is in the fitted dictionary; if
                            # not, skip
                            if context_word in self.vocabulary:
                                X[target_dict_index, self.vocabulary[context_word]] += 1
        
        # if pmi = True, compute pmi matrix from word-context raw frequencies
        # more concise code taken from this StackOverflow post:
        # https://stackoverflow.com/questions/58701337/how-to-construct-ppmi-matrix-from-a-text-corpus
        if self.pmi:
            denom = X.sum()
            col_sums = X.sum(axis = 0)
            row_sums = X.sum(axis = 1)
            
            expected = np.outer(row_sums, col_sums)/denom
            
            X = X/expected
            
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                
                    if X[i,j] > 0:
                        X[i,j] = np.log(X[i,j]) - np.log(self.spmi_k)
                        
                        if self.pmi_positive:
                            X[i,j] = max(X[i,j] - np.log(self.sppmi_k), 0)
        
        # note that X is a dense matrix
        self.X = X

        return X

In [4]:
tweets = pd.read_excel('COVID19_Dataset-CM-ZB-complete with sources.xlsx')

In [5]:
tweets

,Is_Unreliable,Category,Tweet
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government..."
2,1,"1, 4, 9",It was only a matter of time before the mainst...
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...
...,...,...,...
555,0,NaN,BREAKING: Harvard classes will move online sta...
556,0,NaN,Singularity University is hosting a FREE Virtu...
557,0,NaN,Coronavirus: how does it spread and what are t...
558,0,NaN,Stanford just cancelled classes for the rest o...


In [6]:
cm = ContextMatrix(window_size = 15, lowercase = True, lemmatize = True, pmi = True)

In [7]:
word_context_matrix = cm.fit_transform(tweets['Tweet'])

In [8]:
pd.DataFrame(word_context_matrix, index = cm.vocabulary, columns = cm.vocabulary)

,!,#,(,),",",-,--,.,..,...,...,zombie,zone,zoomer,zuckerberg,—,‘,’,“,”,❝real
!,2.531314,-0.503127,0.000000,-1.483746,-1.146376,-0.470562,0.000000,-0.373401,0.00000,1.658890,...,0.000000,0.000000,0.000000,0.0,0.272595,0.000000,0.610607,0.747004,-0.331541,1.738932
#,-0.503127,0.957290,-0.869089,-0.901879,-0.432241,-0.240093,-0.199199,-0.144481,0.52904,-1.342762,...,0.000000,-0.569572,0.223658,0.0,-0.649615,-0.257793,-0.262813,-1.338357,-1.253751,0.000000
(,0.000000,-0.869089,0.080202,2.820001,-0.141311,-0.731163,0.000000,-0.228537,0.00000,1.110606,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.566285,-0.494426,0.000000,0.000000
),-1.483746,-0.901879,2.820001,0.014623,-0.174101,-0.763953,0.000000,-0.098808,0.00000,1.077817,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.599075,-0.527216,0.000000,0.000000
",",-1.146376,-0.432241,-0.141311,-0.174101,0.127678,-0.339572,0.448537,-0.105209,0.00000,-0.289562,...,1.736391,-0.209519,0.000000,0.0,-0.289562,-0.185422,-0.239232,0.120309,-0.200550,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‘,0.000000,-0.257793,0.000000,0.000000,-0.185422,0.000000,0.000000,-0.253229,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.666872,0.000000,0.000000,0.000000
’,0.610607,-0.262813,-0.566285,-0.599075,-0.239232,-0.279038,0.000000,0.029432,0.00000,0.869584,...,0.000000,0.000000,0.000000,0.0,0.000000,1.666872,0.802131,0.730889,0.958596,1.237309
“,0.747004,-1.338357,-0.494426,-0.527216,0.120309,0.485967,1.274076,-0.110018,0.00000,1.229125,...,0.000000,0.000000,0.000000,0.0,1.229125,0.000000,0.730889,1.010387,2.927574,0.000000
”,-0.331541,-1.253751,0.000000,0.000000,-0.200550,0.388252,1.581826,-0.070532,0.00000,1.536875,...,0.000000,0.000000,0.000000,0.0,1.536875,0.000000,0.958596,2.927574,2.319033,0.000000


In [9]:
word_context_matrix.shape

(2327, 2327)

In [10]:
# standard scaling of word context matrix
std_scaler = StandardScaler()
X_std = std_scaler.fit_transform(word_context_matrix)
X_std

array([[ 3.56089853, -1.22884095, -0.29302739, ...,  0.86572301,
        -0.76445189,  4.49908248],
       [-1.06916138,  1.42036886, -1.67065667, ..., -2.32801546,
        -2.22899148, -0.08456301],
       [-0.30147167, -1.89269852, -0.16589521, ..., -1.03553222,
        -0.23794053, -0.08456301],
       ...,
       [ 0.83833406, -2.74395486, -1.07676301, ...,  1.26909443,
         4.41126801, -0.08456301],
       [-0.8073481 , -2.59047869, -0.29302739, ...,  4.20527429,
         3.44485891, -0.08456301],
       [ 2.35185355, -0.31616359, -0.29302739, ..., -0.27831668,
        -0.23794053, -0.08456301]])

In [35]:
myDict = dict()
for i in range(len(cm.vocabulary)):
    key = 'tok_{}'.format(i)
    myDict[key] = X_std[:,i]
myDict

{'tok_0': array([ 3.56089853, -1.06916138, -0.30147167, ...,  0.83833406,
        -0.8073481 ,  2.35185355]),
 'tok_1': array([-1.22884095,  1.42036886, -1.89269852, ..., -2.74395486,
        -2.59047869, -0.31616359]),
 'tok_2': array([-0.29302739, -1.67065667, -0.16589521, ..., -1.07676301,
        -0.29302739, -0.29302739]),
 'tok_3': array([-2.63906486, -1.71958576,  4.16181159, ..., -1.12753532,
        -0.29441691, -0.29441691]),
 'tok_4': array([-2.75575493, -1.35000076, -0.7773128 , ..., -0.26232244,
        -0.89392422, -0.49914642]),
 'tok_5': array([-1.03867181, -0.67008967, -1.45544316, ...,  0.49107685,
         0.33480349, -0.28611593]),
 'tok_6': array([-0.1483947 , -0.52047564, -0.1483947 , ...,  2.23143843,
         2.80628054, -0.1483947 ]),
 'tok_7': array([-1.21012721, -0.72871788, -0.90548408, ..., -0.65624425,
        -0.57320753, -0.42488066]),
 'tok_8': array([-0.06289874,  1.62707711, -0.06289874, ..., -0.06289874,
        -0.06289874, -0.06289874]),
 'tok_9': 

In [42]:
myDict = dict()
myDict['wcm'] = X_std

In [38]:
vocab_list = list(cm.vocabulary.keys())

# pickle vocab list
with open ('tweet_vocab_list', 'wb') as f:
    pickle.dump(vocab_list, f)

In [43]:
# save to matlab file for PCA & ICA decomposition
savemat('wcm_pmi_optPreprocess.mat', myDict)